### Title: Extracting high-fidelity smaller-scale subgraphs of complex networks by edge-reinforced random walk
### Author: Dan Chen and Housheng Su

In [ ]:
import networkx as nx
import numpy as np

In [2]:
# relabel nodes' labels
def relabel_nodes_labels(G):
    mapping = dict(zip(G, range(len(G.nodes()))))
    G = nx.relabel_nodes(G, mapping)
    return G

def load_graph_data(filename):
    G = nx.read_edgelist("./unweighted_networks/" + filename + ".dat")
    G = relabel_nodes_labels(G)
    return G

def ERRWs_subgraph(G, delta_w, num_time_steps, initial_position, target_size):
    # initialize the weight of each edge
    for i, j in G.edges():
        G[i][j]['weight'] = 1.0

    flag = 0
    S = []
    for node in initial_position:
        # print(node)
        S.append(node) 
        for _ in range(num_time_steps):
            # transfer probability
            total_weight = sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node)])
            transition_probs = {neighbor: G[node][neighbor]['weight'] / total_weight for neighbor in G.neighbors(node)}

            next_node = np.random.choice(list(transition_probs.keys()), p=list(transition_probs.values()))
            S.append(next_node)

            # Update the edge weights
            G[node][next_node]['weight'] += delta_w
            G[next_node][node]['weight'] += delta_w

            node = next_node

            if len(set(S))>=target_size:
                flag = 1
                break
        if flag == 1:
            break
    # print(len(set(S)))

    sample_nodes = list(set(S))
    Gs = G.subgraph(sample_nodes)

    return Gs.copy()

In [3]:
filename = ["Metabolic", "Drosophila", "Music", "Airports", "Proteome",\
            "USpowergrid", "Words", "Highvoltage", "Internet", "Enron"]

net_label = 0
G = load_graph_data(filename[net_label])
N = len(G)
print(G)

Graph with 1436 nodes and 4718 edges


In [4]:
num_time_steps0 = 20000
delta_w = 0.01

DC = nx.degree_centrality(G)
sorted_DC = dict(sorted(DC.items(), key=lambda x: x[1], reverse=True))
sorted_indices = list(sorted_DC.keys())

m = 10 
initial_position = list(sorted_indices[:m])
Gs = ERRWs_subgraph(G, delta_w, int(num_time_steps0), initial_position, int(0.5*N))

print(Gs)

Graph with 718 nodes and 2675 edges
